## 8. 다중분류

### 8-1 데이터 불러오기 및 데이터셋 분할

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [7]:
data = pd.read_csv('Fvote.csv')
data.head()

,gender_female,gender_male,region_Chungcheung,region_Honam,region_Others,region_Sudo,region_Youngnam,edu,income,age,score_gov,score_progress,score_intention,vote,parties
0,0,1,0,0,0,0,1,3,3,3,2,2,4.0,1,2
1,0,1,0,0,1,0,0,2,3,3,2,4,3.0,0,3
2,0,1,0,1,0,0,0,1,2,4,1,3,2.8,1,4
3,1,0,0,0,0,1,0,2,1,3,5,4,2.6,1,1
4,0,1,0,0,0,1,0,1,2,4,4,3,2.4,1,1


In [19]:
# 특성치와 레이블 데이터셋 구분
X = data[data.columns[:13]]
y = data[['parties']]

In [20]:
# 훈련데이터, 테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify = y, random_state=42)

---

### 8-2. 기본모델 학습


In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
# 모델에 학습데이터 적용
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [23]:
# 예측 결과를 pred_train에 저장, 정확도 확인
pred_train = model.predict(X_train)
model.score(X_train, y_train)

0.6265822784810127

In [24]:
# test데이터 예측결과 저장
pred_test = model.predict(X_test)
model.score(X_test, y_test)

0.5471698113207547

In [25]:
# 학습데이터 혼동행렬보기
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print('훈련데이터 오차행렬:\n', confusion_train)

훈련데이터 오차행렬:
 [[21  2  3 11]
 [ 1 26  2 11]
 [ 6  2  5  6]
 [ 7  7  1 47]]


In [26]:
# 테스트데이터의 혼동행렬 보기(정분류, 오분류 교차표)
confusion_test=confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬:\n", confusion_test)

테스트데이터 오차행렬:
 [[ 7  2  1  3]
 [ 1  9  1  2]
 [ 1  2  1  2]
 [ 2  5  2 12]]


---

### 8-3. Grid Search

In [28]:
# 그리드서치를 위한 라이브러리
from sklearn.model_selection import GridSearchCV

In [29]:
param = {'C' : [0.001,0.01, 0.1, 1, 10, 100]}

In [30]:
# 로지스틱 모델에 적용
grid_search = GridSearchCV(LogisticRegression(), param, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]},
             return_train_score=True)

In [33]:
# 정확도가 가장 높은 하이퍼 파라미터 및 정확도 제시
print(f'Best param: {grid_search.best_params_}')
print(f'Best Cross-validity Score : {round(grid_search.best_score_,3)}')

Best param: {'C': 0.1}
Best Cross-validity Score : 0.556


In [34]:
# 테스트 데이터 적용, 정확도 결과
print(f'Test set Score: {round(grid_search.best_score_,3)}')

Test set Score: 0.556


---

### 8-4. Random Search

In [36]:
# 랜덤서치를 위한 라이브러리
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [37]:
param = {'C' : randint(low=0.001, high=100)}

In [40]:
random_search = RandomizedSearchCV(LogisticRegression(), param, cv=5,
                                   n_iter = 1000,
                                   return_train_score=True)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=1000,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000187D1B45CD0>},
                   return_train_score=True)

In [43]:
# 정확도가 가장 높은 하이퍼파라미터 및 정확도 제시
print(f"Best Parameter: {random_search.best_params_}")
print(f"Best Cross-validity Score: {round(random_search.best_score_,3)}")

Best Parameter: {'C': 2}
Best Cross-validity Score: 0.544


In [44]:
# 테스트 데이터에 최적 텀색 하이퍼 파라미터 적용 정확도 결과
print("Test set Score: {:.3f}".format(random_search.score(X_test, y_test)))

Test set Score: 0.528
